# PageRank

<pre>
AUTHOR: Samuelm M.H. (samuel.mh@gmail.com)
DATE: 6-Feb-2015
DESCRIPTION: 
    A small study on how to implement efficiently the PageRank algorithm using Python.
</pre>

---

## Dataset

The chosen dataset is a directed graph where nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. This representation is suitable to be transformed into the adjacency matrix required for the PageRank algorithm.

Information:
* URL: http://snap.stanford.edu/data/web-Stanford.html
* Year: 2002
* Nodes: 281.903 (numeration starts at 1)
* Edges: 2.312.497
* Format: id of origin (tabulator) id of destiny (data start at line 5 and there is no tuple duplication). Represents a hyperlink.
* Size (uncompressed): 31,4MB

In [1]:
DATASET_ORIGINAL = "web-Stanford.txt" #Uncompress the .gz file!
NODES = 281903
EDGES = 2312497

### First thoughts

We need to load the dataset into memory as an adjacency matrix. It can be a square matrix having a boolean value in each cell. The way [NumPy](http://www.numpy.org/) implements this is with a dense matrix where each value is a Byte. Let's see how much memory is needed.

\begin{equation*}
\frac{281903^2_{cells}*1_{Byte/cell}}{2^{30}_{B/GB}} = 74,012 GB
\end{equation*}

More than 74GB to store the adjacency matrix!! It doesn't fit in the RAM of my laptop.

---

## Sparse Matrices

It seems reasonable that, if the dataset size is 31,4MB, there must be a representation of it that fits into memory. The key is that a single node is on average connected only to a few nodes (lets say 10). This means there are a lot of false values (not hyperlinks). As false values can be seen as a zeroes, we say the matrix is [sparse](http://en.wikipedia.org/wiki/Sparse_matrix). A good property of a sparse matrices is that their representation is much smaller as their dense equivalent.

There are several libraries to deal with sparse matrices in Python, but the one i've chosen is [scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html)

In [2]:
import numpy as np
from  scipy import sparse

### Loading the dataset

The goal is to keep the adjacency matrix into a Compressed Sparse Row matrix.
According to the [CSR matrix documentation](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

Advantages of the CSR format:
* Efficient arithmetic operations CSR + CSR, CSR * CSR, etc.
* Efficient row slicing
* Fast matrix vector products

So, the efficient row operations is what we need to apply the power method to compute the PageRank values.

Disadvantages of the CSR format
* Slow column slicing operations (consider CSC)
* Changes to the sparsity structure are expensive (consider LIL or DOK)


If we build an empty CSR matrix and try to modify its values we will get a warning like.
<pre>
SparseEfficiencyWarning: 
    Changing the sparsity structure of a csr_matrix is expensive. 
    lil_matrix is more efficient. 
</pre>

The first solution could be using a **Dictionary Of Keys** based sparse matrix and then converting it to a CSR matrix. But it turns out to be slow.

In [3]:
def dataset2dok():
    with open(DATASET_ORIGINAL,'r') as f:
        dokm = sparse.dok_matrix((NODES,NODES),dtype=np.bool)
        for line in f.readlines()[4:]:
            origin, destiny = (int(x)-1 for x in line.split())
            dokm[destiny,origin]=True
    return(dokm.tocsr())

%time dok_m = dataset2dok()

CPU times: user 34.6 s, sys: 256 ms, total: 34.9 s
Wall time: 34.8 s


Another solution could be building the **Compressed Sparse Row** matrix from its indices instead of modifying the cells.

In [4]:
def dataset2csr():
    row = []
    col = []    
    with open(DATASET_ORIGINAL,'r') as f:
        for line in f.readlines()[4:]:
            origin, destiny = (int(x)-1 for x in line.split())
            row.append(destiny)
            col.append(origin)
    return(sparse.csr_matrix(([True]*EDGES,(row,col)),shape=(NODES,NODES)))

%time csr_m = dataset2csr()

CPU times: user 5.7 s, sys: 60 ms, total: 5.76 s
Wall time: 5.74 s


How much memory does it take to store the adjacency (sparse) matrix?

According to the documentation, a CSR matrix is defined by:
* dtype:   data type of the matrix. Not really needed.
* shape:   2-tuple shape of the matrix.
* ndim:    Number of dimensions (this is always 2). Not really needed.
* data:    CSR format data array of the matrix.
* indices: CSR format index array of the matrix.
* indptr:  CSR format index pointer array of the matrix.

In [5]:
import sys

print "The size in memory of the adjacency matrix is {0} MB".format(
    (sys.getsizeof(csr_m.shape)+
    csr_m.data.nbytes+
    csr_m.indices.nbytes+
    csr_m.indptr.nbytes)/(1024.0**2)
)

The size in memory of the adjacency matrix is 12.1022920609 MB


Suming up. We have gone from 74GB to just 12MB by using a sparse matrix. That is pretty good!

### Save/Load a CSR matrix

Despite the fact that 6 seconds is not a huge loading time, it will be nice to store the CSR matrix in a native format so to speed up the loading/saving times.

Here the trick is to store the arrays the matrix is defined by: shape, data, indices, indptr. Furthermore, it is possible to omit the data array taking into consideration that is an array of "True" values of length the number of edges.

In [6]:
def csr_save(filename,csr):
    np.savez(filename,
        nodes=csr.shape[0],
        edges=csr.data.size,
        indices=csr.indices,
        indptr =csr.indptr
    )

def csr_load(filename):
    loader = np.load(filename)
    edges = int(loader['edges'])
    nodes = int(loader['nodes'])
    return sparse.csr_matrix(
        (np.bool_(np.ones(edges)), loader['indices'], loader['indptr']),
        shape = (nodes,nodes)
    )

In [7]:
DATASET_NATIVE = 'dataset-native.npz'
csr_save(DATASET_NATIVE,csr_m)
%time csr = csr_load(DATASET_NATIVE)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 110 ms


---

## PageRank Algorithm

When the PageRank algorithm is taught, the usual way to compute it consists on calculating the Google matrix $A$.

$$ A = \beta M + \frac{1-\beta}{n}\mathbf{e} \cdot \mathbf{e}^T $$

Where:
* $\beta$ is the probability a user follows a link, so $1-\beta$ is the teleportation factor.
* $n$ is the number of elements (nodes).
* $M$ is a matrix built like this:
    1. The starting point is a graph or adjacency matrix $G$ (shape $n \times n$). If there is a link from $i$ to $j$ then the element $G_{j,i}=1$
    2. Normalize the matrix by columns, so the sum of every column is 1.
    3. If there are columns that sum 0, then set their elements to $1/n$.
* $\mathbf{e}$ is a vector of 1 of length $n$.

And then apply the [**Power Method**](http://en.wikipedia.org/wiki/Power_iteration):

$$ \vec{r}_{(t+1)} = A \cdot \vec{r}_{(t)} $$

Until it converges:

$$  \lVert  \vec{r}_{(t+1)} - \vec{r}_{(t)}  \rVert _1 < \epsilon $$

Then, $\vec{r}$ is an eigenvector of $A$ with the PageRank scores.
A property of this vector is:

$$ \sum^{n}_{i=1} r_i  = 1 $$

This approach has a huge drawback since it has to compute the Google matrix $A$, and, as seen before it is not likely to fit in memory.

### PageRank with a sparse matrix

However, it is possible to rewrite the algorithm in order to work with the sparse adjacency matrix $G$ and save a lot of memory.
The trick consist on computing the Google matrix per row in each iteration in order to save memory and pay CPU cycles.

Here is the pseudocode.

- $\vec{r} \leftarrow 1/n$
- $flag \leftarrow True$
- while $flag$
    - $\vec{r}_{(t+1)}  \leftarrow  G \cdot  \frac{\vec{r}_t \cdot \beta}{\mathbf{d}_{out}} $
    - $\vec{r}_{(t+1)}  \leftarrow \vec{r}_{(t+1)} + \frac{1-\vec{r}_{(t+1)}}{n} \quad$ (Re-insert leaked PageRank)
    - $flag \leftarrow  \lVert  \vec{r}_{(t+1)} - \vec{r}_{(t)}  \rVert _1 < \epsilon \quad$ (Stop condition)
    - $\vec{r}_{(t)}  \leftarrow \vec{r}_{(t+1)}$
    
Where:
- $G$ is the adjacency/connectivity matrix.
- $\mathbf{d}_{out}$ is a vector with the out-degree values of the nodes.

It is also possible to compute the value of $\mathbf{d}_{out} \cdot \frac{1}{\beta}$ since it is an invariant inside the loop.

### Implementation

In [8]:
def compute_PageRank(G, beta=0.85, epsilon=10**-4):
    '''
    Efficient computation of the PageRank values using a sparse adjacency 
    matrix and the iterative power method.
    
    Parameters
    ----------
    G : boolean adjacency matrix. np.bool8
        If the element j,i is True, means that there is a link from i to j.
    beta: 1-teleportation probability.
    epsilon: stop condition. Minimum allowed amount of change in the PageRanks
        between iterations.

    Returns
    -------
    output : tuple
        PageRank array normalized top one.
        Number of iterations.

    '''    
    #Test adjacency matrix is OK
    n,_ = G.shape
    assert(G.shape==(n,n))
    #Constants Speed-UP
    deg_out_beta = G.sum(axis=0).T/beta #vector
    #Initialize
    ranks = np.ones((n,1))#np.ones((n,1))/n #vector
    time = 0
    flag = True
    while flag:        
        time +=1
        with np.errstate(divide='ignore'): # Ignore division by 0 on ranks/deg_out_beta
            new_ranks = G.dot((ranks/deg_out_beta)) #vector
        #Leaked PageRank
        new_ranks += (1-new_ranks.sum())/n#(1-new_ranks.sum())/n
        #Stop condition
        if (np.linalg.norm(ranks-new_ranks,ord=1)<=epsilon):
            flag = False        
        ranks = new_ranks
    return(ranks, time)

### Adjency Matrix G

In [6]:
#Graph Orange
row = np.array([1, 2, 3, 0, 1, 2, 3])
col = np.array([0, 1, 2, 3, 3, 3, 3])
data = np.array([1, 1, 1, .25, .25, .25, .25])
G = sparse.csr_matrix((data, (row, col)), shape=(4, 4))
G.toarray()

array([[ 0.  ,  0.  ,  0.  ,  0.25],
       [ 1.  ,  0.  ,  0.  ,  0.25],
       [ 0.  ,  1.  ,  0.  ,  0.25],
       [ 0.  ,  0.  ,  1.  ,  0.25]])

In [77]:
#Graph Green
row = np.array([1, 2, 2, 0])
col = np.array([0, 0, 1, 2])
data = np.array([1, 1, 1, 1])
G = sparse.csr_matrix((data, (row, col)), shape=(3, 3))
G.toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [1, 1, 0]])

### Experiment

In [10]:
print '==> Computing PageRank'
%time pr,iters = compute_PageRank(G, beta=0.1)
print '\nIterations: {0}'.format(iters)
print 'Element with the highest PageRank: {0}'.format(np.argmax(pr)+1)
print pr, '\n'
print 'a+b= {0}'.format(pr[[0]]+pr[[1]]) # a+b
print 'a+c= {0}'.format(pr[[0]]+pr[[2]]) # a+c

==> Computing PageRank
CPU times: user 3.5 ms, sys: 826 µs, total: 4.32 ms
Wall time: 3.7 ms

Iterations: 18
Element with the highest PageRank: 4
[[ 0.12207504]
 [ 0.21972718]
 [ 0.29784656]
 [ 0.36035123]] 

a+b= [[ 0.34180221]]
a+c= [[ 0.41992159]]


In [79]:
a=pr[[0]]
b=pr[[1]]
c=pr[[2]]

print 3*(a+b)
print 3*(a+c)

[[ 1.8077335]]
[[ 2.35558826]]


In [80]:
print .95*a == .9*c + .05*b
print a == c + .15*b
print b == .575*a + .15*c
print a == .9*c + .05*b

[[False]]
[[False]]
[[False]]
[[False]]


In [81]:
print .95*a
print a
print b
print a

[[ 0.36838522]]
[[ 0.38777392]]
[[ 0.21480391]]
[[ 0.38777392]]


In [82]:
print .9*c + .05*b
print c + .15*b
print .575*a + .15*c
print .9*c + .05*b

[[ 0.36842015]]
[[ 0.42964276]]
[[ 0.28258333]]
[[ 0.36842015]]


## Bibliography

- Page, Lawrence; Brin, Sergey; Motwani, Rajeev y Winograd, Terry (1999). [The PageRank citation ranking: Bringing order to the Web.](http://ilpubs.stanford.edu/422/1/1999-66.pdf)